In [ ]:
### RUN IN COLAB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Find the weights
# https://drive.google.com/file/d/1hnSWoQnG8Bnne3GiDXF-A_Nnwc5EhqSf/view?usp=share_link

In [ ]:
!git clone https://github.com/Mafaz03/Stable-diffusion.git

Cloning into 'Stable-diffusion'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 171 (delta 64), reused 148 (delta 41), pack-reused 0 (from 0)
Receiving objects: 100% (171/171), 6.22 MiB | 20.04 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
import os
os.chdir('Stable-diffusion')

In [ ]:
!pip install -q diffusers
!pip install -q pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 54.6 MB/s eta 0:00:00


In [ ]:
os.chdir('sd')

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader
import config
from config import initial_transforms
from dataset import ImageDataset
import torch
from ddpm import DDPMSampler
import torch
from pipeline import rescale
from encoder import VAE_Encoder
from decoder import VAE_Decoder
import model_converter
from clip import CLIP
from utils import generate_context
from pipeline import get_time_embedding
from transformers import CLIPTokenizer
from torch.nn import functional as F
from diffusers.optimization import get_cosine_schedule_with_warmup
import gc


from diffusion import Diffusion
diffusion = Diffusion()
print("DIffuser model successfully initilised for training!")

generator = torch.Generator(device=config.DEVICE)
noise_scheduler = DDPMSampler(generator=generator)
print("DDPM model successfully initialised!")

clip = CLIP()
print("CLIP model successfully Loaded")


state_dict = model_converter.load_from_standard_weights("/content/drive/MyDrive/v1-5-pruned-emaonly.ckpt", config.DEVICE)

# Initialize the decoder
decoder = VAE_Decoder().to(config.DEVICE)
decoder.load_state_dict(state_dict['decoder'], strict=True)
print("Decoder model successfully Loaded")

# Initialize the encoder
encoder = VAE_Encoder().to()
encoder.load_state_dict(state_dict['encoder'], strict=True)  # This should load into the encoder
print("Encoder model successfully Loaded")

tokenizer = CLIPTokenizer("../data/vocab.json", merges_file="../data/merges.txt")
print("Tokensizer successfully Loaded")

train_ds = ImageDataset("dataset/Train", initial_transforms, "dataset/discription.json")
train_dl = DataLoader(train_ds, batch_size=config.TRAIN_BATCH_SIZE, shuffle=True)

test_ds = ImageDataset("dataset/Test", initial_transforms, "dataset/discription.json")
test_dl = DataLoader(train_ds, batch_size=config.EVAL_BATCH_SIZE, shuffle=True)

print("Dataset loaded Successfull!")

optimizer = torch.optim.AdamW(diffusion.parameters(), lr=config.LEARNING_RATE)
lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.LR_WARMUP_STEPS,
    num_training_steps=(len(train_dl) * config.EPOCHS),
)

print("Optimizer loaded and lr scheduler set!")

DIffuser model successfully initilised for training!
DDPM model successfully initialised!
CLIP model successfully Loaded
Decoder model successfully Loaded
Encoder model successfully Loaded
Tokensizer successfully Loaded
Dataset loaded Successfull!
Optimizer loaded and lr scheduler set!


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from torch.cuda.amp import autocast, GradScaler

scaler = torch.amp.GradScaler('cuda')

In [ ]:
import torch
import torch.nn.functional as F

losses = []

# Ensure all variables are on the same device
batch = next(iter(train_dl))
clean_images = batch[0].to(config.DEVICE)  # Move to GPU
timestep = torch.randint(0, noise_scheduler.num_train_timesteps, (1,), device=config.DEVICE)  # Correct device
bs = clean_images.shape[0]
latents_shape = (bs, 4, config.LATENT_SIZE, config.LATENT_SIZE)
input_image_tensor = rescale(clean_images, (0, 255), (-1, 1)).to(config.DEVICE)  # Correct device
encoder_noise = torch.randn(latents_shape, generator=generator, device=config.DEVICE)  # Correct device
encoder = encoder.to(config.DEVICE)
latents = encoder(input_image_tensor, encoder_noise)  # Already on the correct device

# Add noise
noise, noisy_latents = noise_scheduler.add_noise(latents, timestep)

# Generate context and time embedding
context = generate_context(tokenizer, clip, batch[1])
context = torch.cat([context] * bs).to(config.DEVICE)  # Correct device
time_embedding = get_time_embedding(timestep).to(config.DEVICE)  # Correct device


gc.collect()
torch.cuda.empty_cache()

with torch.amp.autocast("cuda"):
  diffusion = diffusion.to(config.DEVICE)
  noise_pred_latent = diffusion(latents, context, time_embedding)
  noise = noise.to(config.DEVICE)
  loss = F.mse_loss(noise_pred_latent, noise)

# Scale the loss and backpropagate
scaler.scale(loss).backward()

# Update weights
scaler.step(optimizer)
scaler.update()

diffusion.to(config.IDLE_DEVICE)
losses.append(loss.item())

gc.collect()
torch.cuda.empty_cache()



RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 2 but got size 1 for tensor number 1 in the list.

In [ ]:
print(f"Latents shape: {latents.shape}")  # Check latents shape
print(f"Context shape: {context.shape}")  # Check context shape
print(f"Time embedding shape: {time_embedding.shape}")

Latents shape: torch.Size([4, 4, 4, 4])
Context shape: torch.Size([20, 77, 768])
Time embedding shape: torch.Size([1, 320])


In [ ]:
diffusion(latents, context, time_embedding)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 2 but got size 1 for tensor number 1 in the list.

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!python sd/train.py